민원(콜센터) 질의응답_금융보험_사고 및 보상 문의_Training.json

{
		"도메인" : "금융/보험" ,
		"카테고리" : "사고 및 보상 문의" ,
		"대화셋일련번호" : "A20437" ,
		"화자" : "상담사" ,
		"문장번호" : "10" ,
		"고객의도" : "" ,
		"상담사의도" : "건강보험 산정특례 재등록" ,
		"QA" : "A" ,
		"고객질문(요청)" : "" ,
		"상담사질문(요청)" : "" ,
		"고객답변" : "" ,
		"상담사답변" : "네. 재검사에서 다시 확진받은 경우라면 병원이 달라도 상관없습니다." ,
		"개체명 " : "병원" ,
		"용어사전" : "병원/장소" ,
		"지식베이스" : "병원,장소" 
	}

[RAG 자료 생성을 위한 정제]

1) 대화 단위로 그룹화: 대화셋일련번호를 기준으로 각 대화들을 하나로 묶습니다.

2) 시나리오(Context) 생성: 그룹화된 각 대화의 전체 내용을 시간 순서(문장번호)에 따라 하나의 텍스트로 결합합니다. 이 텍스트가 바로 RAG 모델이 참조할 **컨텍스트(시나리오)**가 됩니다.

3) 질문-답변 쌍 추출: 각 대화 시나리오에서 고객의 핵심 질문과 그에 대한 상담사의 최종 답변을 추출하여 **질문(Question)**과 답변(Answer) 쌍을 만듭니다.

4) 데이터 구조화 및 저장: 위에서 생성된 scenario, question, answer 각각의 컬럼으로 하는 새로운 데이터 프레임을 만들어 CSV 파일로 저장

In [ ]:
import json
import pandas as pd

# Load the JSON file
with open('민원(콜센터) 질의응답_금융보험_사고 및 보상 문의_Training.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Get unique conversation IDs
conversation_ids = df['대화셋일련번호'].unique()

rag_data = []

# Process each conversation
for conv_id in conversation_ids:
    # Filter by conversation ID and sort by sentence number
    conversation_df = df[df['대화셋일련번호'] == conv_id].sort_values(by='문장번호')

    # --- 1. Create the full scenario (context) ---
    scenario = []
    for index, row in conversation_df.iterrows():
        speaker = row['화자']
        # Combine all text fields for a complete dialogue turn
        customer_q = row['고객질문(요청)']
        agent_q = row['상담사질문(요청)']
        customer_a = row['고객답변']
        agent_a = row['상담사답변']

        text = ""
        if pd.notna(customer_q) and customer_q:
            text = customer_q
        elif pd.notna(agent_q) and agent_q:
            text = agent_q
        elif pd.notna(customer_a) and customer_a:
            text = customer_a
        elif pd.notna(agent_a) and agent_a:
            text = agent_a

        if text:
            scenario.append(f"{speaker}: {text}")

    full_scenario = "\n".join(scenario)

    # --- 2. Extract Question and Answer ---
    # Find the first customer question as the main question
    first_question_row = conversation_df[conversation_df['고객질문(요청)'].notna() & (conversation_df['고객질문(요청)'] != '')]
    question = first_question_row['고객질문(요청)'].iloc[0] if not first_question_row.empty else None

    # Concatenate all agent answers to form the comprehensive answer
    agent_answers = conversation_df[conversation_df['상담사답변'].notna() & (conversation_df['상담사답변'] != '')]['상담사답변']
    answer = "\n".join(agent_answers) if not agent_answers.empty else None

    # Add to our RAG data list if we have a valid Q&A pair and context
    if question and answer and full_scenario:
        rag_data.append({
            'scenario': full_scenario,
            'question': question,
            'answer': answer
        })

# Create the final DataFrame
rag_df = pd.DataFrame(rag_data)

# Save the DataFrame to a JSON file
# orient='records' creates a list of dictionaries, one for each row.
# force_ascii=False ensures Korean characters are saved correctly.
# indent=4 makes the JSON file readable.
json_output_path = 'refined_insurance_rag_data.json'
rag_df.to_json(json_output_path, orient='records', force_ascii=False, indent=4)


# Display the path to the created file and a sample of the data
print(f"JSON 파일이 '{json_output_path}' 경로에 저장되었습니다.")
print("\n저장된 JSON 데이터 샘플 (첫 2개):")
print(rag_df.head(2).to_json(orient='records', force_ascii=False, indent=4))

대화 시나리오 중 '보험' 또는 '보상' 이라는 키워드가 포함된 시나리오 4200개의 데이터,2884개의 데이터가 해당 조건을 만족

scenario 필드에 '보험'과 '보상' 키워드가 모두 포함된 대화 시나리오를 추출
= 총 291개의 데이터가 해당 조건을 만족

In [ ]:
import json
import pandas as pd

# Load the previously created JSON file
with open('refined_insurance_rag_data.json', 'r', encoding='utf-8') as f:
    rag_data = json.load(f)

# Filter the data
filtered_data = [item for item in rag_data if '보험' in item['scenario']]

# Save the filtered data to a new JSON file
with open('filtered_insurance_data.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

# Print the count of original and filtered data
print(f"원본 데이터 개수: {len(rag_data)}")
print(f"필터링 후 데이터 개수: {len(filtered_data)}")
print("\nJSON 파일이 성공적으로 생성되었습니다: filtered_insurance_data.json")

# Display a sample of the filtered data
print("\n필터링된 데이터 샘플 (상위 2개):")
print(json.dumps(filtered_data[:2], ensure_ascii=False, indent=4))

    {
        "scenario": "고객: 보험금 청구 하려고요?\n상담사: 메뉴중에 보상이라고 적인 거  누르시면 보험금 청구 페이지가 있습니다. \n고객: 잠깐만요..아 찾았어요 \n상담사: 네 거기에 청구하실 내용 안내대로 입력하시면 됩니다. \n고객: 영수증들은  스캔해야 하나요?\n상담사: 아니요, 스캔하셔도 되고. 그냥 사진 찍어서 등록하셔도 됩니다.\n고객: 네\n고객: 근데, 처리하는데는 얼마나 걸리죠?\n상담사: 심사 시작되면 일주일 내로 입금됩니다. \n고객: 어디로 돈이 들어오죠?\n상담사: 고객님이 등록해주신 통장으로 입금됩니다.\n상담사: 네 어떤 상품에 가입하셨죠?\n고객: 네, 고맙습니다.\n고객: 실비보험요\n상담사: 네, 혹시 현대해상 어플 있으신가요?\n고객: 네 깔려 있어요.\n상담사: 그럼 로그인 하시고 보험금청구 페이지에 들어가시면 됩니다. \n고객: 잘 못찾겠는데요?\n상담사: 화면 우측상단에 점3개 보이시죠? 거기 누르시면 됩니다.\n고객: 네 눌렀는데.. 그 다음에는요?",
        "question": "보험금 청구 하려고요?",
        "answer": "메뉴중에 보상이라고 적인 거  누르시면 보험금 청구 페이지가 있습니다. \n네 거기에 청구하실 내용 안내대로 입력하시면 됩니다. \n아니요, 스캔하셔도 되고. 그냥 사진 찍어서 등록하셔도 됩니다.\n심사 시작되면 일주일 내로 입금됩니다. \n고객님이 등록해주신 통장으로 입금됩니다.\n그럼 로그인 하시고 보험금청구 페이지에 들어가시면 됩니다. \n화면 우측상단에 점3개 보이시죠? 거기 누르시면 됩니다."
    }